In [1]:
import sys

from pprint import pprint
from typing import Dict, List, Tuple, Optional
import os
import numpy as np
import timeit
import statistics
import matplotlib.pyplot as plt
import re
from scipy.special import factorial

In [2]:
def read_from_file_1(file: str = "data1.txt"):
    D = []
    f = open(file, "r")
    for row in f:
        data = row.strip().split(' ')
        D += data
    f.close()
    return np.array(D).astype(np.int64)

In [3]:
def count_increases(D) -> int:
    increases = 0
    for i in range(0,D.shape[0]-1):
        if D[i]<D[i+1]:
            increases += 1
    return increases

In [4]:
def count_increases_sliding_window(D) -> int:
    increases = 0
    for i in range(0,D.shape[0]-3):
        if (D[i]+D[i+1]+D[i+2])<(D[i+1]+D[i+2]+D[i+3]):
            increases += 1
    return increases

In [5]:
def solve_1():
    D = read_from_file_1()
    print("The solution for 1.1 is:",count_increases(D))
    print("The solution for 1.2 is:",count_increases_sliding_window(D))

In [6]:
def location(file: str = "data2.txt"):
    D = []
    f = open(file, "r")
    hor = 0
    depth = 0
    for row in f:
        data = row.strip().split(' ')
        pos_change = int(data[1])
        if data[0] == "forward":
            hor += pos_change
        elif data[0] == "down":
            depth += pos_change
        else:
            depth -= pos_change
            
    
    return hor, depth

In [7]:
def location_with_aim(file: str = "data2.txt"):
    D = []
    f = open(file, "r")
    hor = 0
    depth = 0
    aim = 0
    for row in f:
        data = row.strip().split(' ')
        pos_change = int(data[1])
        if data[0] == "forward":
            hor += pos_change
            depth += pos_change * aim
        elif data[0] == "down":
            aim += pos_change
        else:
            aim -= pos_change
            
    return hor, depth, aim

In [8]:
def solve_2():
    hor, depth = location("data2.txt")
    mult1 = hor*depth
    print("The solution for 2.1 is:",mult1)
    hor, depth, aim = location_with_aim("data2.txt")
    mult2 = hor*depth
    print("The solution for 2.2 is:",mult2)

In [9]:
def read_from_file_3(file: str = "data3.txt"):
    D = []
    f = open(file, "r")
    for row in f:
        data = row.strip().split()
        data[0] = list(data[0])
        D += data
    f.close()
    return np.array(D).astype(np.int64)

In [10]:
def convert_to_bin(A):
    lenght = len(A)
    return [A[i] * (2**(lenght-i-1)) for i in range(0,lenght)]

In [11]:
def decode(D):
    lenght = D.shape[1]
    gamma_rate = np.zeros(lenght)
    epsilon_rate = np.zeros(lenght)
    for i in range(0, lenght):
        if np.mean(D,axis=0)[i] > 0.5:
            gamma_rate[i] = 1  
        else:
            epsilon_rate[i] = 1
    gamma_rate = convert_to_bin(gamma_rate)
    epsilon_rate = convert_to_bin(epsilon_rate)
    gamma_epsilon = np.sum(gamma_rate)*np.sum(epsilon_rate)
    return gamma_epsilon
            

In [12]:
def oxygen(D):
    i = 0
    while D.shape[0] > 1:
        if np.mean(D,axis=0)[i] >= 0.5:
            indexes = np.argwhere(D[:,i]==1).flatten()
        else:
            indexes = np.argwhere(D[:,i]==0).flatten()
        D = D[indexes,:]
        i += 1
    oxygen = np.sum(convert_to_bin(D.flatten()))
    return oxygen

In [13]:
def co2(D):
    i = 0
    while D.shape[0] > 1:
        if np.mean(D,axis=0)[i] < 0.5:
            indexes = np.argwhere(D[:,i]==1).flatten()
        else:
            indexes = np.argwhere(D[:,i]==0).flatten()
        D = D[indexes,:]
        i += 1
    co2 = np.sum(convert_to_bin(D.flatten()))
    return co2

In [14]:
def solve_3():
    D = read_from_file_3("data3.txt")
    gamma_epsilon = int(decode(D))
    mult = oxygen(D)*co2(D)
    print("The solution for 3.1 is:",gamma_epsilon)
    print("The solution for 3.2 is:",mult)

In [15]:
def read_from_file_4(file: str = "data4.txt"):
    draws = []
    boards = []
    f = open(file, "r")
    draws = f.readline().strip().split(',')
    board = np.zeros(shape=(5,5))
    i = 0
    for row in f:
        if i == 5:
            i = 0
            boards.append(board.astype(np.int64))
        if row == '\n':
            board = np.zeros(shape=(5,5))
            continue
        board[i,:] = row.strip().split()
        i += 1
    boards.append(board.astype(np.int64))
    f.close()
    return np.array(draws).astype(np.int64), boards

In [16]:
def findwinners(boards):
    counter = len(boards)
    winners = []
    for i in range(0,counter):
        for row in boards[i]:
            if np.all(row==-1) and i not in winners:
                winners.append(i)
                continue
        for column in boards[i].T:
            if np.all(column==-1) and i not in winners:
                winners.append(i)
                continue
    return winners

In [17]:
def play_to_win(file: str = "data4.txt"):
    draws, boards = read_from_file_4(file)
    index_of_winning_board = -1
    for draw in draws:
        for board in boards:
            pos = np.where(board == draw)
            if pos != []:
                board[pos] = -1
        indexes_of_winning_boards = findwinners(boards)
        if indexes_of_winning_boards != []:
            winning_board = boards[indexes_of_winning_boards[0]]
            winning_draw = draw
            break
    
    return winning_board, winning_draw

In [18]:
def calc_score(board, winning_draw):
    indexes = np.where(board == -1)
    board[indexes] = 0
    return np.sum(board) * winning_draw

In [19]:
def play_to_lose(file: str = "data4.txt"):
    draws, boards = read_from_file_4(file)
    for draw in draws:
        for board in boards:
            pos = np.where(board == draw)
            if pos != []:
                board[pos] = -1
        indexes_of_winning_boards = findwinners(boards)
        if len(boards) == 1 and indexes_of_winning_boards != []:
            return boards[0], draw
        if indexes_of_winning_boards != []:
            for i in reversed(indexes_of_winning_boards):
                boards.pop(i)      
        
    return -1, -1

In [20]:
def solve_4():
    winning_board, winning_draw = play_to_win("data4.txt")
    losing_board, losing_draw = play_to_lose("data4.txt")
    winning_score = calc_score(winning_board, winning_draw)
    losing_score = calc_score(losing_board, losing_draw)
    print("The solution for 4.1 is:",winning_score)
    print("The solution for 4.2 is:",losing_score)

In [21]:
class Coordinate:
    def __init__(self, x_1,y_1,x_2,y_2)->None:
        self.x1 = x_1
        self.y1 = y_1
        self.x2 = x_2
        self.y2 = y_2
    
    def get_all_points_on_lines(self):
        points = []
        if self.x1 != self.x2 and self.y1 != self.y2:
            if self.x1<self.x2:
                if self.y1<self.y2:
                    p = []
                    q = []
                    leng = self.x2+1 - self.x1
                    for i in range(0, leng):
                        p.append(self.x1+i)
                        q.append(self.y1+i)
                    points = [p,q]
                else:
                    p = []
                    q = []
                    leng = self.x2+1 - self.x1
                    for i in range(0, leng):
                        p.append(self.x1+i)
                        q.append(self.y1-i)
                    points = [p,q]
            elif self.x1>self.x2:
                if self.y1<self.y2:
                    p = []
                    q = []
                    leng = self.x1+1 - self.x2
                    for i in range(0, leng):
                        p.append(self.x1-i)
                        q.append(self.y1+i)
                    points = [p,q]
                else:
                    p = []
                    q = []
                    leng = self.x1+1 - self.x2
                    for i in range(0, leng):
                        p.append(self.x1-i)
                        q.append(self.y1-i)
                    points = [p,q]
                
        else:   
            if self.y1 < self.y2:
                p = [self.x1]
                q = []
                for i in range(self.y1, self.y2+1):
                    q.append(i)
                points = [p,q]
            elif self.y1 > self.y2:
                p = [self.x1]
                q = []
                for i in range(self.y2, self.y1+1):
                    q.append(i)
                points = [p,q]
            elif self.x1 < self.x2:
                p = []
                q = [self.y1]
                for i in range(self.x1, self.x2+1):
                    p.append(i)
                points = [p,q]
            else:
                p = []
                q = [self.y1]
                for i in range(self.x2, self.x1+1):
                    p.append(i)
                points = [p,q]
        return points

In [22]:
def read_from_file_5(file: str = "data5.txt"):
    coordinates = []
    largest_x = 0
    largest_y = 0
    f = open(file, "r")
    for row in f:
        data = re.split(',|->',row.strip())
        co = Coordinate(int(data[0]),int(data[1]),int(data[2]),int(data[3]))
        if int(data[0])>largest_x: largest_x = int(data[0])
        if int(data[2])>largest_x: largest_x = int(data[2]) 
        if int(data[1])>largest_y: largest_y = int(data[1]) 
        if int(data[3])>largest_y: largest_y = int(data[3])
        coordinates.append(co)
        
    f.close()
    return coordinates, largest_x, largest_y

In [23]:
def get_diagram(coordinates, largest_x, largest_y):
    diagram = np.zeros(shape=(largest_x+1, largest_y+1))
    for curr in coordinates:
        if curr.x1 != curr.x2 and curr.y1 != curr.y2:
            continue
        diagram[curr.get_all_points_on_lines()] += 1
    return diagram

In [24]:
def get_danger(diagram):
    indexes = np.where(diagram < 2)
    diagram[indexes] = 0
    indexes = np.where(diagram > 1)
    diagram[indexes] = 1
    return int(np.sum(diagram))

In [25]:
def get_diagram_with_diag(coordinates, largest_x, largest_y):
    diagram = np.zeros(shape=(largest_x+1, largest_y+1))
    for curr in coordinates:
        #print(curr.get_all_points_on_lines())
        diagram[curr.get_all_points_on_lines()] += 1
    return diagram

In [26]:
def solve_5():
    coordinates, largest_x, largest_y = read_from_file_5("data5.txt")
    di = get_diagram(coordinates, largest_x, largest_y)
    print("The solution for 5.1 is:",get_danger(di))
    di_diag = get_diagram_with_diag(coordinates, largest_x, largest_y)
    print("The solution for 5.2 is:",get_danger(di_diag))

In [27]:
def read_from_file_6(file: str = "data6.txt"):
    fish = []
    f = open(file, "r")
    for row in f:
        fish = row.strip().split(',')
        fish = [int(f1) for f1 in fish]
    f.close()
    return fish

In [28]:
def sim_lanternfish(num_of_days, fish):
    x = 0
    for i in range(0,num_of_days):
        new = 0
        for f in range(0,len(fish)):
            if fish[f] == 0:
                new += 1
                fish[f] = 6
            else:
                fish[f] -=1
        for i in range(0,new):
            fish.append(8)
    return fish

In [29]:
def write_array(fish):
    numpy_fish = np.zeros(shape=(9))
    for f in fish:
        numpy_fish[f] += 1
    return numpy_fish

In [30]:
def sim_lanternfish_well(num_of_days, fish_long):
    fish = write_array(fish_long)
    for i in range(0,num_of_days):
        changing_fish = fish[0]
        fish[0:6] = fish[1:7]
        fish[6] = changing_fish + fish[7]
        fish[7] = fish[8]
        fish[8] = changing_fish
    return fish
        

In [31]:
def solve_6():
    fish1 = read_from_file_6("data6.txt")
    print("The solution for 6.1 is:",len(sim_lanternfish(80,fish1)))
    fish2 = read_from_file_6("data6.txt")
    x = sim_lanternfish_well(256, fish2)
    print("The solution for 6.2 is:",int(np.sum(x)))

In [32]:
def read_from_file_7(file: str = "data7.txt"):
    crabs = np.array(())
    f = open(file, "r")
    for row in f:
        crabs_ = row.strip().split(',')
        crabs = np.array(crabs_).astype(np.int64)
    f.close()
    return crabs

In [33]:
def find_alignment(crabs):
    lowest = np.min(crabs)
    highest = np.max(crabs)
    lowest_cost = (np.Inf,np.Inf)
    for i in range(lowest,highest+1):
        cost = np.sum(abs(crabs - i))
        if cost<lowest_cost[0]:
            lowest_cost = (int(cost),i)
    return lowest_cost

In [34]:
def find_alignment_expensive(crabs):
    lowest = np.min(crabs)
    highest = np.max(crabs)
    lowest_cost = (np.Inf,np.Inf)
    for i in range(lowest,highest+1):
        n= abs(crabs - i)
        cost = np.sum(n*(n+1)/2)
        if cost<lowest_cost[0]:
            lowest_cost = (int(cost),i)
    return lowest_cost

In [35]:
def solve_7():
    crabs = read_from_file_7("data7.txt")
    al1 = find_alignment(crabs)
    al2 = find_alignment_expensive(crabs)
    print("The solution for 7.1 is:",al1[0])
    print("The solution for 7.2 is:",al2[0])

In [36]:
def read_from_file_8(file: str = "data8.txt"):
    signal_pattern = []
    output = []
    f = open(file, "r")
    for row in f:
        line = row.strip().split('|')
        signal_pattern_ = line[0].split()
        for s in range(0,len(signal_pattern_)):
            signal_pattern_[s] = set(signal_pattern_[s])
        signal_pattern.append(signal_pattern_)
        output_ = line[1].split()
        for s in range(0,len(output_)):
            output_[s] = set(output_[s])
        output.append(output_)
                      
    f.close()
    return np.array(signal_pattern),np.array(output)

In [37]:
def find_1478(output):
    valid_numbers = [2,3,4,7]
    count = 0
    for i in range(0,len(output)):
        if len(output[i]) in valid_numbers:
            count +=1
    return count

In [38]:
def decode_all_seven_digits(signal_pattern,output):
    decoded_output = 0
    for i in range(0,signal_pattern.shape[0]):
        decoded_output_ = decode_seven_digits(signal_pattern[i], output[i])
        decoded_output += decoded_output_
    return decoded_output

In [39]:
def decode_seven_digits(pattern, out):
    not_decoded = True
    sorted_digits = np.array([{},{},{},{},{},{},{},{},{},{}])
    for p in pattern: #found 1,3,4,8
        if len(p) == 2:
            sorted_digits[1] = p
        elif len(p) == 3:
            sorted_digits[7] = p
        elif len(p) == 4:
            sorted_digits[4] = p
        elif len(p) == 7:
            sorted_digits[8] = p
    for p in pattern: #found 6,9,0
        if len(p) == 6 and len(p & sorted_digits[1])==1:
            sorted_digits[6] = p
        elif len(p) == 6 and len((sorted_digits[4] | sorted_digits[7]) & p)==5:
            sorted_digits[9] = p
        elif len(p) == 6:
            sorted_digits[0] = p
    for p in pattern: #found 2,3,5
        if len(p) == 5 and (sorted_digits[6] & sorted_digits[9])==p:
            sorted_digits[5] = p
        elif len(p) == 5 and len(p & (sorted_digits[6] & sorted_digits[9])) ==3:
            sorted_digits[2] = p
        elif len(p) == 5:
            sorted_digits[3] = p 
    
    number = find_seven_digit_value(sorted_digits, out)
    
    return number

In [40]:
def find_seven_digit_value(sorted_digits, out):
    number = 0
    for o in range(0,len(out)):
        for i in range(0,len(sorted_digits)):
            if sorted_digits[i] == out[o]:
                number += 10**(len(out)-o-1) * i
                break
                
    return number

In [41]:
def solve_8():
    signal_pattern, output = read_from_file_8("data8.txt")
    print("The solution for 8.1 is:",find_1478(output.flatten()))
    print("The solution for 8.2 is:",decode_all_seven_digits(signal_pattern,output))

In [42]:
def read_from_file_9(file: str = "data9.txt"):
    floor_map = []
    f = open(file, "r")
    for row in f:
        line = row.strip().split()
        line = [int(char) for char in line[0]]
        floor_map.append(line)  
    f.close()
    return np.array(floor_map)

In [43]:
def find_lowest_locations(floor_map):
    max_len_y = floor_map.shape[1]
    max_len_x = floor_map.shape[0]
    zero = []
    one = []
    for ix, iy in np.ndindex(floor_map.shape):
        num_of_comparisons = 0
        num_of_lowest_points = 0
        if ix-1 > -1:
            num_of_comparisons +=1
            if floor_map[ix-1,iy]>floor_map[ix,iy]:
                num_of_lowest_points +=1
        if iy-1 > -1:
            num_of_comparisons +=1
            if floor_map[ix,iy-1]>floor_map[ix,iy]:
                num_of_lowest_points +=1
        if ix+1 < max_len_x:
            num_of_comparisons +=1
            if floor_map[ix+1,iy]>floor_map[ix,iy]:
                num_of_lowest_points +=1
        if iy+1 < max_len_y:
            num_of_comparisons +=1
            if floor_map[ix,iy+1]>floor_map[ix,iy]:
                num_of_lowest_points +=1
        if num_of_lowest_points/num_of_comparisons == 1:
            zero.append(ix)
            one.append(iy)
    return [tuple(zero),tuple(one)]        

In [44]:
def get_risk_level(floor_map,locations):    
    return np.sum(floor_map[locations[0],locations[1]]) + len(locations[0])

In [45]:
def get_basin_size(floor_map,locx,locy):
    x = [locx]
    y = [locy]
    xy = [(locx,locy)]
    max_len_y = floor_map.shape[1]
    max_len_x = floor_map.shape[0]
    not_done = True
    while not_done:
        num_of_changes = 0
        for i in range(0,len(x)):
            if x[i]-1 > -1:
                if floor_map[x[i]-1,y[i]]!=9 and not (x[i]-1,y[i]) in xy:
                    xy.append((x[i]-1,y[i]))
                    x.append(x[i]-1)
                    y.append(y[i])
                    num_of_changes +=1
            if y[i]-1 > -1:
                if floor_map[x[i],y[i]-1]!=9 and not (x[i],y[i]-1) in xy:
                    xy.append((x[i],y[i]-1))
                    x.append(x[i])
                    y.append(y[i]-1)
                    num_of_changes +=1
            if x[i]+1 < max_len_x:
                if floor_map[x[i]+1,y[i]]!=9 and not (x[i]+1,y[i]) in xy:
                    xy.append((x[i]+1,y[i]))
                    x.append(x[i]+1)
                    y.append(y[i])
                    num_of_changes +=1
            if y[i]+1 < max_len_y:
                if floor_map[x[i],y[i]+1]!=9 and not (x[i],y[i]+1) in xy:
                    xy.append((x[i],y[i]+1))
                    x.append(x[i])
                    y.append(y[i]+1)
                    num_of_changes +=1
        if num_of_changes == 0:
            not_done = False
    return len(xy)  

In [46]:
def find_basins(floor_map,locations):
    sizes = np.array([0,0,0])
    for loc in range(0,len(locations[0])):
        curr_size = get_basin_size(floor_map,locations[0][loc],locations[1][loc])
        sizes = np.sort(sizes, axis=None)
        if curr_size > sizes[0]:
            sizes[0] = curr_size
    return np.prod(sizes)

In [47]:
def solve_9():
    floor_map = read_from_file_9("data9.txt")
    locations = find_lowest_locations(floor_map)
    risk_level = get_risk_level(floor_map,locations)
    biggest_basins = find_basins(floor_map,locations)
    print("The solution for 9.1 is:",risk_level)
    print("The solution for 9.2 is:",biggest_basins)

In [48]:
def read_from_file_10(file: str = "data10.txt"):
    chunks = []
    f = open(file, "r")
    for row in f:
        line = row.strip().split()
        line = [char for char in line[0]]
        chunks.append(line)  
    f.close()
    return chunks

In [49]:
def find_corrupt(chunk):
    stack = []
    opening = ["(","[","{","<"]
    find_open = {
        ")": "(",
        "]": "[",
        "}": "{",
        ">": "<"
    }
    for char in chunk:
        if char in opening:
            stack.append(char)
        elif find_open[char] == stack[-1] and len(stack)!=0:
            stack.pop()
        else:
            return char
    return ""

In [50]:
def find_corrupt_lines(chunks):
    para_value = {
        ")": 3,
        "]": 57,
        "}": 1197,
        ">": 25137
    }
    i = 0
    indexes = []
    sum_of_corruption = 0
    for chunk in chunks:
        char = find_corrupt(chunk)
        if char == "":
            indexes.append(i)
            i += 1
            continue
        sum_of_corruption += para_value[char]
        i += 1
        
    return sum_of_corruption, indexes

In [51]:
def complete_chunk(chunk):
    stack = []
    opening = ["(","[","{","<"]
    find_open = {
        ")": "(",
        "]": "[",
        "}": "{",
        ">": "<"
    }
    find_closed = {
        "(": ")",
        "[": "]",
        "{": "}",
        "<": ">"
    }
    find_score = {
        ")": 1,
        "]": 2,
        "}": 3,
        ">": 4
    }
    ending = []
    score = 0
    for char in chunk:
        if char in opening:
            stack.append(char)
        elif find_open[char] == stack[-1] and len(stack)!=0:
            stack.pop()
            
    for i in range(0,len(stack)):
        curr_open = stack.pop()
        ending.append(find_closed[curr_open])
        score *= 5
        score += find_score[find_closed[curr_open]]
    return ending, score

In [52]:
def complete_other_lines(chunks,indexes_of_not_corrupt):
    new_chunks = []
    for i in range(0,len(chunks)):
        if i in indexes_of_not_corrupt:
             new_chunks.append(chunks[i])
    chunks = new_chunks
    full_score = []
    for chunk in chunks:
        _, score = complete_chunk(chunk)
        full_score.append(score)
    full_score.sort()
    index = int(len(full_score)/2)
    return full_score[index]

In [53]:
def solve_10():
    chunks = read_from_file_10("data10.txt")
    sum_of_corruption, indexes = find_corrupt_lines(chunks)
    full_score = complete_other_lines(chunks,indexes)
    print("The solution for 10.1 is:",sum_of_corruption)
    print("The solution for 10.2 is:",full_score)

In [54]:
def read_from_file_11(file: str = "test11.txt"):
    octo = []
    f = open(file, "r")
    for row in f:
        line = row.strip().split()
        line = [int(char) for char in line[0]]
        octo.append(line)  
    f.close()
    return np.array(octo)

In [93]:
def simulate(octo):
    total_number_of_glows = 0
    for i in range(0,100):
        octo = octo + 1
        number_of_glows, new_octo = change_glowing_levels(0,octo)
        total_number_of_glows += number_of_glows
        indexes = np.where(new_octo==-1)
        if len(indexes[0]) == 100:
            return i
        new_octo[indexes] = 0
        octo = new_octo
    return total_number_of_glows

In [94]:
def add_one_to_neighbors(octo,indexes_of_glowing):
    x = indexes_of_glowing[0]
    y = indexes_of_glowing[1]
    for i in range(0,len(indexes_of_glowing[0])):
        if x[i]-1 > -1 and octo[x[i]-1,y[i]] != -1:
            octo[x[i]-1,y[i]] += 1
        if x[i]-1 > -1 and y[i]-1 > -1 and octo[x[i]-1,y[i]-1] != -1:
            octo[x[i]-1,y[i]-1] += 1
        if x[i]-1 > -1 and y[i]+1 < 10 and octo[x[i]-1,y[i]+1] != -1:
            octo[x[i]-1,y[i]+1] += 1
        if y[i]-1 > -1 and octo[x[i],y[i]-1] != -1:
            octo[x[i],y[i]-1] += 1
        if x[i]+1 < 10 and octo[x[i]+1,y[i]] != -1:
            octo[x[i]+1,y[i]] += 1
        if x[i]+1 < 10 and y[i]-1 > -1 and octo[x[i]+1,y[i]-1] != -1:
            octo[x[i]+1,y[i]-1] += 1
        if x[i]+1 < 10 and y[i]+1 < 10 and octo[x[i]+1,y[i]+1] != -1:
            octo[x[i]+1,y[i]+1] += 1
        if y[i]+1 < 10 and octo[x[i],y[i]+1] != -1:
            octo[x[i],y[i]+1] += 1             
    return octo    

In [97]:
def change_glowing_levels(number_of_glows,octo):
    indexes_of_glowing = np.where(octo>9)
    octo = add_one_to_neighbors(octo,indexes_of_glowing)
    octo[indexes_of_glowing] = -1 
    if len(np.where(octo>9)[0]) == 0:
        return number_of_glows+len(indexes_of_glowing[0]),octo
    return change_glowing_levels(len(indexes_of_glowing[0])+number_of_glows,octo)

In [95]:
def simulate_all_flashing(octo):
    total_number_of_glows = 0
    for i in range(0,500):
        octo = octo + 1
        number_of_glows, new_octo = change_glowing_levels(0,octo)
        total_number_of_glows += number_of_glows
        indexes = np.where(new_octo==-1)
        if len(indexes[0]) == 100:
            return i+1
        new_octo[indexes] = 0
        octo = new_octo
    return 0

In [98]:
octo = read_from_file_11("data11.txt")
print(simulate_all_flashing(octo))

329


In [ ]:
solve_1()
solve_2()
solve_3()
solve_4()
solve_5()
solve_6()
solve_7()
solve_8()
solve_9()
solve_10()